
# Multiple inheritances

Unlike other object-oriented programming languages like Java and C#, C++ supports multiple inheritances. That means, in C++, a class can inherit the public and protected members of more than one class. For example, class `C` below inherits from both classes `A` and `B`. This is depicted in the following class hierarchy:

```text
    A                B
    △               △
    |                |
    |                |
    +------- C ------+
```

In [1]:
#include <iostream>
using namespace std;

In [ ]:
class A { // Base class 1
};

class B { // Base class 2
};

class C : public A, public B { //Derived class from A and B
};

### CODING CHALLENGE
Define the classes that implement the following hierarchy with Manager and Scientist each inheriting from both Employee and Student.

```text
    Employee                      Student
        △                           △
        |                            |
        |                            |
        +--------- Manager ----------+
        |                            |
        |                            |
        +---------Scientist ---------+
```

In [ ]:
//TODO
class Employee {
};

class Student {
};

class Manager : public Employee, public Student {
};

class Scientist : public Employee, public Student {
};

There is, however, a reason why other languages do not support multiple inheritances; it can lead to overcomplicated program designs and suffers from a set of problems, one of which is the infamous **diamond probelm**. Here is an example of such a problem. 

```text
        +--------▷ Parent ◁ --------+    
        |                            |
        |                            |
      Child1                      Child2 
        △                           △
        |                            |
        |                            |
        +-------- GrandChild --------+
```    

In [ ]:
class Parent {
public:
   void foo(){}
};

class Child1 : public Parent {
public:
   void bar(){}
};

class Child2 : public Parent {};
class GrandChild : public Child1, public Child2 {};

The problem here is that `Grandchild` has two copies of `foo()`: one inherited from `Parent` via `Child1` and another, also inherited from `Parent`, via `Child2`. So when you call `foo()` on a `GrandChild` object, which one will be used? Let's see how the compiler reacts to this situation. Uncomment the following code to see the errors.

In [ ]:

GrandChild d;
d.foo();


To the compiler, `d.foo()` is an ambiguous call and it does not know which version of `foo()` to use. It is up to us, the programmers, to clear this up for the compiler and we do that in one of two ways:
* Tell the compiler exactly which version of `foo()` to use using the class name and the resolution operator `::`. For example, the code below, tells the compiler to use the version of `foo()` inherited by `GrandChild` via `Child2`.

In [ ]:
GrandChild gc;
gc.Child2::foo();

* Or use the keyword `virtual` to instruct the compiler to prevent creating multiple copies of functions of the same `Parent`. This means changing the implementation of the above class hierarchy to something like this:

In [ ]:
class AnotherParent {
public:
   void foo(){}
};

class AnotherChild1 : virtual public AnotherParent {
public:
   void bar(){}
};

class AnotherChild2 : virtual public AnotherParent {};
class AnotherGrandChild : public AnotherChild1, public AnotherChild2 {};

Notice the use of the keyword `virtual` before the public inheritance of the `AnotherChild1` and `AnotherChild2` classes. This tells C++ that if another class inherits from both of these classes (like `AnotherGrandChild` here), only single copies of the functions of the base class `AnotherParent` are made available to it. Thus eliminating the previous ambiguity.

Having done that allows us to run the following without the use of the resolution operator `::`

In [ ]:
AnotherGrandChild d2;
d2.foo();

### CODING CHALLENGE

Define the classes that implement the following class hierarchy. Use the keyword `virtual` to insure your classes do not suffer from the **diamond problem**.

```text
        +---------▷ BaseClass ◁ ---------+    
        |                                 |
        |                                 |
  DerivedClass1                     DerivedClass2 
        △                                △
        |                                 |
        |                                 |
        +----- DerivedFurtherClass1 ------+
        |                                 |
        |                                 |
        +----- DerivedFurtherClass2 ------+
``` 

In [ ]:
//TODO

class BaseClass {
public: 
    void poo(){}
};

class DerivedClass1 : virtual public BaseClass{
public: 
    void par(){}
};

class DerivedClass2 : virtual public BaseClass{
};

class DerivedFurtherClass1 : public DerivedClass1, public DerivedClass2 {};
class DerivedFurtherClass2 : public DerivedClass1, public DerivedClass2 {};


# Virtual Functions

In addition to encapsulation and inheritance, **polymorphism** is the third principle of object-oriented programming. It means multiple forms. and it is supported by C++ at both compile-time and run-time. At compile-time, we can think of function overloading and operator overloading as forms of polymorphism, where the same function name or operator is used to mean and do different things in different contexts. At run-time polymorphism is achieved by combining inheritance and **virtual functions**.

Generally speaking **virtual** means existing in appearance but not in reality. In c++, a **virtual function** is a member function declared at the base class and redefined (or overridden) by a derived class. When this happens, the derived class will have access to two functions with the same name and signature: one provided by the base class and the other by the derived class itself. When a **virtual function** is called, the compiler defers deciding on which function definition to use until run-time. We call this **late binding** which is different from the **early binding** utilized by non-virtual functions.

To understand **virtual functions** and the difference between **early binding** and **late binding**, let's look at an example. First here is our base class:

In [ ]:
class Base {
public:
    void show() { cout << "Base\n"; }
};

Next let's create two "regular" classes that derive from it. Both of these classes override the `show()` function. No `virtual` functions yet.

In [ ]:
class Derived1 : public Base {
public:
    void show() { cout << "Derived 1\n"; }
};

class Derived2 : public Base {
public:
    void show() { cout << "Derived 2\n"; }
};

Using these classes, we can create the following objects:

In [ ]:
Derived1 d1; 
Derived2 d2_1, d2_2;

Now we call the `show()` functions on these objects and study the output:

In [ ]:
d1.show();
d2_1.show();
d2_2.show();

These calls are behaving as expected, with each calling the version of `show()` defined in its own class.

Let's now see what happens when `Base` pointers are used.

In [ ]:
Base* bptr; 
bptr = &d1; 
bptr->show();
bptr = &d2_1; 
bptr->show();

Here the `show()`  function of the `Base` class is called instead of the ones from the derived classes. This  is inspite of this  pointer `bptr` pointing to a `Derived1` object first and a `Derived2` object after that. This is example of **early binding** with the compiler determining which version of `show()` to use at compile-time based on the type of the pointer, which is `Base`.

Let's try the same thing with a `Base` reference:

In [ ]:
Base& bref = d2_2;
bref.show();

**Early binding** happens here as well. 

To enable **late binding** we need to use **virtual functions**. Here is an example of virtual functions starting with the base class:

In [ ]:
class AnotherBase {
public:
    virtual void show() { cout << "Another Base\n"; }
    virtual ~AnotherBase(){} // Virtual destructor
};

Notice the use of the keyword `virtual` in front of the `show()` function. This makes this function **virtual** and instructs the compiler to enable **late binding** on it. 

Notice the adding  of the virtual destructor. The destructor should be maked virtual on any class with one ore more virtual functions. This is to make sure objects of derived classes are destroyed properly. This applies **polymorphism** and **late binding** to object cleanup and makes sure that the correct destructors based on the actual objects are called when objects are deleted. 

Let's define two derived classes of this base class:

In [ ]:
class AnotherDerived1 : public AnotherBase {
public:
    void show() { cout << "Another Derived 1\n"; }
};

class AnotherDerived2 : public AnotherBase {
public:
    void show() { cout << "Another Derived 2\n"; }
};


Notice that we did not use the keyword `virtual` inside these derived class; only inside the base. This is because a function needs only be defined as `virtual` once and in the base class. After that, no matter how many times a it is  inherited, it remains virtual.

Let's now see what **virtual functions** do, starting with the following objects:

In [ ]:
AnotherDerived1 ad1; 
AnotherDerived2 ad2_1, ad2_2;

First, let's call the `show()` function using the **dot operator**.

In [ ]:
ad1.show();
ad2_1.show();
ad2_2.show();

This worked as expected. Let's see what happens when `Base` pointers are used. 

In [ ]:
AnotherBase* abptr; 
abptr = &ad1; 
abptr->show();
abptr = &ad2_1; 
abptr->show();

Now the correct `show()` function is called based on the actual object. This is **late binding** in action. 

Let's try the same thing, but  with a `Base` reference:

In [ ]:
AnotherBase& abref = ad2_2;
abref.show();

The correct `show()` function is being called and **late binding** worked here as well.

To summarize, A **virtual function**:
* is a member function that is declared as `virtual' in a base class and redefined (or overridden) by one or more derived classes.
* works like a *normal* non-virtual function when accessed by the **dot operator**.
* supports **late binding** as opposed to **early binding** when accessed via a pointer or a reference.

### CODING CHALLENGE
**PART 1**: Define a class named `Rectangle` with four data members:
* two integers x and y for the coordinates of the upper left corner.
* two integers width and height for its width and height.

This class should have a constructor that takes four arguments one for each of its data members. It should also have a virtual function named `getName()` that returns the string `"Rectangle"` and a virtual destructor.

**PART 2**: Knowing that a square is a rectangle whose width and height are the same, define a class names `Square` that inherits from `Rectangle` This class should have a constructor that takes three arguments (x, y, and width) and delegates initialization to the `Rectangle` constructor. It should also override the `getName()` function by returning the string `"Square"`.

In [2]:
//TODO: The Rectangle class

class Rectangle{
    private:
        int x;
        int y;
        int width;
        int height;
    public: 
        Rectangle(int x, int y, int w, int h): x(x), y(y), width(w), height(h){}
    virtual string getName(){
        return "Rectangle";
    }
        virtual ~Rectangle(){}
};

In [3]:
//TODO: The Square class

class Square: public Rectangle{
    private:
        int x;
        int y;
        int width;
    public:
        Square(int x, int y, int w): Rectangle(x, y, w, w){}
    
    string getName(){
        return "Square";
    }
}

### CODING CHALLENGE

Create two objects: a rectangle and a square. Also, create a rectangle pointer named `rptr` and initialize it to `nullptr`. Then:

* point the `rptr` pointer to the rectangle object and use it to call the `getName()` function.
* point the `rptr` pointer to the square object and use it to call the `getName()` function.

In [4]:
//TODO // HELP

Square s(4, 4, 4);
Rectangle r(4, 3, 2, 1);
Rectangle* rptr = nullptr;
    
rptr = &r;
cout << rptr->getName() << endl;
rptr = &s;
cout << rptr->getName() << endl;

Rectangle
Square


### CODING CHALLENGE

Create another two objects: a rectangle and a square. Create a rectangle reference named `rref` and make it an alias to the square object. Use it to call the `getName()` function.

In [5]:
//TODO

Square s2(4, 4, 4);
Rectangle r2(4, 3, 2, 1);

Rectangle& rref = s2;

cout << rref.getName() << endl;

Square


## Pure functions

Finally, there is the notion of **pure virtual functions**, which are defined by having the `= 0` at the end of their prototypes. A **pure virtual function** is a function that has no actual definition within the class that declares it; only the prototype is provided. This makes any class declaring one or more pure function **an abstract class**. which is a class whose implementation is not complete and therefore cannot be used to create objects. 

Here is an example abstract class.

In [ ]:
class SomeClass {
public:
    virtual void print() = 0; // a pure function
};

Having a **pure virtual function** makes this class **abstract**, which means it is not complete (or concrete) and cannot, therefore, be used to create objects. For example, the following statement throws a compilation error. Run it to see these errors.

In [ ]:
SomeClass sc;

To summarize, **abstract classes** are meant to be subclassed or inherited by other classes. Their **pure functions** are like holes that make them incomplete and unfit to be used for creating objects.